In [ ]:
!pip install transformers datasets peft trl accelerate bitsandbytes -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login("")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

data = {
    "prompt": [
        "What is the capital of France?",
        "Explain the water cycle.",
        "Who wrote Hamlet?",
    ],
    "response": [
        "The capital of France is Paris.",
        "The water cycle describes how water moves through the Earth: evaporation, condensation, precipitation, and collection.",
        "William Shakespeare wrote Hamlet."
    ]
}

dataset = Dataset.from_dict(data)


In [ ]:
def preprocess(batch):
    texts = [
        f"QUESTION: {q}\nANSWER: {a}"
        for q, a in zip(batch["prompt"], batch["response"])
    ]
    tokenized = tokenizer(texts, max_length=256, truncation=True, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = dataset.map(preprocess, batched=True, remove_columns=["prompt", "response"])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# Apply PEFT / LoRA
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_proj","k_proj","v_proj","o_proj"],
    lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Training using transformers.Trainer
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=50,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="./mistral-finetuned",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=lambda data: {
        "input_ids": torch.stack([torch.tensor(f["input_ids"]) for f in data]),
        "attention_mask": torch.stack([torch.tensor(f["attention_mask"]) for f in data]),
        "labels": torch.stack([torch.tensor(f["labels"]) for f in data])
    }
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()
trainer.save_model("./mistral-finetuned")
tokenizer.save_pretrained("./mistral-finetuned")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,8.803400
2,8.803400
3,8.803400
4,8.803400
5,8.803400
6,8.803400
7,8.803400
8,5.678900
9,2.188900
10,0.525100


('./mistral-finetuned/tokenizer_config.json',
 './mistral-finetuned/special_tokens_map.json',
 './mistral-finetuned/tokenizer.model',
 './mistral-finetuned/added_tokens.json',
 './mistral-finetuned/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained("./mistral-finetuned", quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("./mistral-finetuned")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
input_text = "### Prompt:\nWho wrote Hamlet?\n\n### Response:\n"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

output_ids = model.generate(input_ids, max_new_tokens=50)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### Prompt:
Who wrote Hamlet?

### Response:
William Shakespeare wrote Hamlet.

### Rationale:
William Shakespeare wrote Hamlet.

### Sources:
- [Wikipedia](https://en.wikipedia.org/wiki/William_Shakespeare)


In [ ]:
!zip -r mistral-finetuned.zip ./mistral-finetuned


  adding: mistral-finetuned/ (stored 0%)
  adding: mistral-finetuned/tokenizer.json (deflated 85%)
  adding: mistral-finetuned/tokenizer.model (deflated 55%)
  adding: mistral-finetuned/adapter_config.json (deflated 56%)
  adding: mistral-finetuned/README.md (deflated 66%)
  adding: mistral-finetuned/tokenizer_config.json (deflated 68%)
  adding: mistral-finetuned/adapter_model.safetensors (deflated 7%)
  adding: mistral-finetuned/runs/ (stored 0%)
  adding: mistral-finetuned/runs/Jun13_04-24-26_8324229db175/ (stored 0%)
  adding: mistral-finetuned/runs/Jun13_04-24-26_8324229db175/events.out.tfevents.1749788673.8324229db175.786.0 (deflated 62%)
  adding: mistral-finetuned/checkpoint-50/ (stored 0%)
  adding: mistral-finetuned/checkpoint-50/adapter_config.json (deflated 56%)
  adding: mistral-finetuned/checkpoint-50/README.md (deflated 66%)
  adding: mistral-finetuned/checkpoint-50/rng_state.pth (deflated 25%)
  adding: mistral-finetuned/checkpoint-50/optimizer.pt (deflated 8%)
  adding

In [ ]:
from google.colab import files
files.download("mistral-finetuned.zip")
